<h3> HANDY PYTHON PACKAGES </h3>

In [215]:
import requests
import json
import urllib
import os
from random import randint
import pandas as pd
from IPython.display import display
%matplotlib inline

<H3> PREREQUISITES</H3>

In [116]:
api_key = '' #write you SOCIB api_key
api_url = 'http://api.socib.es'
headers = {
    'accept': 'application/vnd.socib+json; version=latest',#'application/vnd.socib+json', #application/vnd.socib+json; version=latest
    'api_key': api_key,
}

<H3> AUXILIARY FUNCTIONS </H3>

In [212]:
def params_concatenation(params, values):
    """
    Concatenates the params and their values in an unique string to later be attached to the generic API ENDPOINTS
    """
    params_concatenation = ''
    for param, value in zip(params,values):
        if params.index(param) != len(PARAMS)-1:
            params_concatenation = params_concatenation + param+'='+value+'&'
        else:
            params_concatenation = params_concatenation + param+'='+value
    return params_concatenation;

<h3> DATA-SOURCES/ID/DATA: STRAIGHT FORWARD DATA ACCESS</h3>

As we already know from:
<ul>
    <li>[Quick start](https://github.com/pazrg/SOCIB_API/blob/master/tips/quick_start.ipynb)</li>
    <li>[finding_your_data_source](https://github.com/pazrg/SOCIB_API/blob/master/data_sources/finding_your_data_source.ipynb)</li>
    <li>[working_with_data_sources_netcdfs](https://github.com/pazrg/SOCIB_API/blob/master/working_with_data_sources_netcdfs.ipynb)</li>
</ul>

SOCIB API provides:
<ul>
    <li>information about the available deployments and netCDFs by the /DATA-SOURCES/ ENDPOINT</li>
    <li>detailed information about a given deployment by the /DATA-SOURCES/ID/</li>
    <li>filtering capabilities over the retourned ones by playing with the so-called PARAMS</li>
</ul>

Apart from that, SOCIB API also facilitates a quick and agnostic (not need to deal with netCDF format) access to the deployment data. This is accomplished by a new ENDPOINT: /DATA-SOURCES/ID/DATA/. <br>
Let's dive into this new ENPOINT.

Making a simple query to the /DATA-SOURCE/ ENDPOINT: 

In [117]:
end_point = '/data-sources/'
request = requests.get('%s%s' % (api_url, end_point), headers=headers)
response = json.loads(request.text)

print('Requested url: '+'%s%s' % (api_url, end_point))
print('SOCIB has performed so far a total of %s deployments'%(response['count']))

Requested url: http://api.socib.es/data-sources/
SOCIB has performed so far a total of 666 deployments


Choosing randomly one of the above deployments and asking for its data:

In [124]:
random = randint(0,response['count']-1) #random deployment
print(random)

205


In [225]:
page = random/8
item = (random - (random/8)*8)-1
print('Deployment number %s is on page %s at position %s:'%(random,page,item))

Deployment number 205 is on page 25 at position 4:


In [233]:
#requesting the page where the deployment is
end_point = '/data-sources/'
random_request = requests.get('%s%s?%s' % (api_url, end_point, 'page='+str(page)), headers=headers)
random_response = json.loads(random_request.text)

#selecting inside this page the desired deployment
random_deployment = random_response['results'][item] 

print('- Data-source with id %s is composed by %s netCDF files:' %(random_deployment['id'], len(deployment['entries'])))
for netCDF in deployment['entries']:
    netCDF_name = netCDF['services']['opendap']['url'].split('/')[10]
    print('   '+ netCDF_name)
print(' being operative since %s to %s'%(random_deployment['initial_datetime'],random_deployment['end_datetime']))

- Data-source with id 0cacc73faa is composed by 2 netCDF files:
   dep0004_buoy-canaldeibiza_scb-met010_L1_2017-02.nc
   dep0004_buoy-canaldeibiza_scb-met010_L0_2017-02.nc
 being operative since 2015-10-19T11:17:39Z to 2015-12-11T08:00:42Z


If we want to know the data contained within an specific deployment, we can either rely on THREDDS (see <a href="working_with_data_sources_netcdfs" targter="_blank">working_with_data_sources_netcdfs</a>) or asking directly the API for it by providing the id of such deployment. From example, from the above query whe know data-sources id:

In [230]:
request = requests.get('%s%s%s/data/' % (api_url, end_point, random_deployment['id']), headers=headers)
print('%s%s%s/data/' % (api_url, end_point, random_deployment['id']))
response = json.loads(request.text)
values = []
cols = ['processing level','data type','coordinates','variables','time coordinate']
index = []
numb = 0
for proc_level in response.keys():
    for feature in response[proc_level]['features']:
        vals = []
        for var in feature['variables']:
            vals.append(var+' (expressed as '+','.join(feature['variables'][var].keys())+')')
        values.append([proc_level,feature['feature_type'],','.join(feature['coordinates'].keys()),'<br>'.join(vals),feature['coordinates']['time']['data'][0]+'-<br>'+feature['coordinates']['time']['data'][len(feature['coordinates']['time']['data'])-1]])
        index.append(random_deployment['id']+'<span style="display:none">'+str(numb)+'</span>')
        numb = numb +1
df = pd.DataFrame(values, index=index, columns=cols)   
df.style

http://api.socib.es/data-sources/0cacc73faa/data/


This way, variables (here stated by its standard names) are associated to specific parameters (within parentheses) so that it is easier to know which parameter(s) represent(s) such variable. All those parameters that are still under study (are they practical? are they redundant?) at SOCIB are grouped temporary under a <i>fake</i> standard name: _NONE_. 

The defaulting time window displayed when queriying the data of a specific variable is the last 30 days of the deployment data. In order to access a different time period or even obtain some kind of preprocessed data, there are a number of PARAMS to add to the above DATA-SOURCES/ID/DATA ENPOINT:

<ul><li>initial_datetime</li> 
    <ul>UTC time as YYY-MM-DDTHH:MM:SS</ul>
</ul>
<ul><li>end_datetime</li> 
    <ul>UTC time as YYY-MM-DDTHH:MM:SS</ul>
</ul>
<ul><li>standard_variable</li> 
    <ul>Any value of those returned by <a href="https://github.com/pazrg/SOCIB_API/blob/master/tips/quick_start.ipynb" target="_blank">/standard-variables/ ENDPOINT</a></ul>
</ul>
<ul><li>bbox</li>
    <ul>Area where a given deployment has been operating since deployed (released or placed). Area should be specify as a 4 comma-separated float numbers following the structure: min. lat., max. lat., min. lon., max. lon</ul>
</ul>
<ul><li>elevation_range</li>
    <ul>2 comma-separated float following the structure: min. elevation, max. elevation</ul>
</ul>
<ul><li>processing_level</li>
    <ul>L1 (with qc variables/flags) or L0 (raw data)</ul>
</ul>
<ul><li>feature_type?</li>
    <ul>Any value of those returned by <a href="https://github.com/pazrg/SOCIB_API/blob/master/tips/quick_start.ipynb" target="_blank">/data-types/ ENDPOINT</a></ul>
</ul>
<ul><li>resample_rule</li>
    <ul>The time frequency to be used in order to resample the result data. The values of this parameter handled by Pandas library. This full values list can be found in the <a href="http://pandas.pydata.org/pandas-docs/stable/timeseries.html#offset-aliases" target="_blank">pandas time offset aliases docs(s)</a>. Some examples would be: how=1d, how=1w or how=1AS</ul>
</ul>
<ul><li>resample_how</li> 
    <ul>The method to be used in order to _resample_and_interpolate the result data. The values of this parameter handled by Pandas library. This possible values of this parameter are: sum, mean, std, sem, max, min, median, first, last, ohlc.</ul>
</ul>
<ul><li>max_qc_value</li>
    <ul>Provide only the values of each variable whose corresponding QC value is better or equal (less or equal) than the given value. The values whose QC values are greater than the given value will be switched by null.</ul>
</ul>

Let's see some examples of these PARAMS:

In [237]:
#just one hour of data
end_point = '/data-sources/'+random_deployment['id']+'/data/'
PARAMS = ['standard_variable','initial_datetime', 'end_datetime']
PARAMS_values = ['sea_water_salinity','2015-10-19T12:00:00','2015-10-19T13:00:00' ]
                
#concatenation of PARAMS
param_concatenation = params_concatenation(PARAMS,PARAMS_values)
a = '%s%s?%s' % (api_url, end_point,param_concatenation) 
request_prof = requests.get(a, headers=headers)
response_prof = json.loads(request_prof.text)
ids = []
for key in response_prof.keys():
    for feature in response_prof[key]['features']:
        print('- With processing level %s and feature_type %s:'%(key, feature['feature_type']))
        print('Standard variable %s can be found expressed as:' %(PARAMS_values[0]))
        for item in feature['variables'][PARAMS_values[0]].keys():
            ids.append(item)
            df = pd.DataFrame(feature['variables'][PARAMS_values[0]][item].values(), index=feature['variables'][PARAMS_values[0]][item].keys(), columns=[item])
            display(df.style)    

- With processing level L2 and feature_type trajectoryProfile:
Standard variable sea_water_salinity can be found expressed as:


- With processing level L1 and feature_type trajectory3d:
Standard variable sea_water_salinity can be found expressed as:


In [238]:
#just the data with qc flags and masking the data with qc>1
end_point = '/data-sources/'+random_deployment['id']+'/data/'
PARAMS = ['standard_variable','initial_datetime', 'end_datetime','processing_level','max_qc_value']
PARAMS_values = ['sea_water_salinity','2015-10-19T12:00:00','2015-10-19T13:00:00','L1','1' ]

#concatenation of PARAMS
param_concatenation = params_concatenation(PARAMS,PARAMS_values)
a = '%s%s?%s' % (api_url, end_point,param_concatenation) 
request_prof = requests.get(a, headers=headers)
response_prof = json.loads(request_prof.text)
ids = []
for key in response_prof.keys():
    for feature in response_prof[key]['features']:
        print('- With processing level %s and feature_type %s:'%(key, feature['feature_type']))
        print('Standard variable %s can be found expressed as:' %(PARAMS_values[0]))
        for item in feature['variables'][PARAMS_values[0]].keys():
            ids.append(item)
            df = pd.DataFrame(feature['variables'][PARAMS_values[0]][item].values(), index=feature['variables'][PARAMS_values[0]][item].keys(), columns=[item])
            display(df.style)    

- With processing level L1 and feature_type trajectory3d:
Standard variable sea_water_salinity can be found expressed as:


In [242]:
#every 5 minutes mean of the above data
end_point = '/data-sources/'+random_deployment['id']+'/data/'
PARAMS = ['standard_variable','initial_datetime', 'end_datetime','processing_level','max_qc_value','resample_rule','resample_how']
PARAMS_values = ['sea_water_salinity','2015-10-19T12:00:00','2015-10-19T13:00:00','L1','1','5min','mean' ]

#concatenation of PARAMS
param_concatenation = params_concatenation(PARAMS,PARAMS_values)
a = '%s%s?%s' % (api_url, end_point,param_concatenation) 
request_prof = requests.get(a, headers=headers)
response_prof = json.loads(request_prof.text)
ids = []
for key in response_prof.keys():
    for feature in response_prof[key]['features']:
        print('- With processing level %s and feature_type %s:'%(key, feature['feature_type']))
        print('Standard variable %s can be found expressed as:' %(PARAMS_values[0]))
        for item in feature['variables'][PARAMS_values[0]].keys():
            ids.append(item)
            df = pd.DataFrame(feature['variables'][PARAMS_values[0]][item].values(), index=feature['variables'][PARAMS_values[0]][item].keys(), columns=[item])
            display(df.style)

- With processing level L1 and feature_type trajectory3d:
Standard variable sea_water_salinity can be found expressed as:
